In [2]:
import pandas as pd
from pandas import ExcelWriter
import os

# Папка с исходным файлом и куда будут сохраняться результаты
file_path = "data/2024.xlsx"
base_folder = "data"

# Проверка: есть ли папка, если нет — создать
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

# Загружаем Excel-файл
xls = pd.ExcelFile(file_path)

# Оставляем только листы, которые начинаются с цифры
relevant_sheets = [name for name in xls.sheet_names if name[0].isdigit()]

# Ключевые слова для распределения таблиц
keywords = {
    "1_Производство": ["производство продукции животноводства"],
    "2_Численность": ["численность скота", "численность птицы"],
    "3_Молоко": ["производство молока"],
    "4_Яйца": ["производство яиц"],
    "5_Забой": ["забито", "реализовано на убой"]
}

# Найдём подходящие листы
sheet_matches = {k: [] for k in keywords}
for sheet in relevant_sheets:
    try:
        preview = xls.parse(sheet, nrows=2).astype(str).fillna("").values.astype(str)
        joined_text = " ".join(preview.flatten()).lower()
        for key, kw_list in keywords.items():
            if any(kw in joined_text for kw in kw_list):
                sheet_matches[key].append(sheet)
    except Exception:
        continue

# Сохраняем в отдельные файлы
for file_label, sheets in sheet_matches.items():
    output_path = os.path.join(base_folder, f"{file_label}.xlsx")
    with ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for sheet in sheets:
            try:
                df = xls.parse(sheet)
                df.to_excel(writer, index=False, sheet_name=sheet[:31])
            except Exception as e:
                print(f"Ошибка при сохранении листа {sheet}: {e}")
